In [1]:
import glob
import ezdxf
import json
import numpy as np

In [2]:
thesis_path = r"C:\Users\cvaka\OneDrive\Master\Thesis"

In [3]:
def dxf_to_dict(file):
    dxf_dict = {}
    doc = ezdxf.readfile(file)
    msp = doc.modelspace()
    for e in msp:
        if e.dxftype() == "LINE":
            points = [e.dxf.start[:2], e.dxf.end[:2]]
        elif e.dxftype() == "LWPOLYLINE":
            points = e.get_points('xy')
        else:
            print('Dxftype {} not supported, hence ignored'.format(e.dxftype()))

        if e.dxf.layer in dxf_dict:
            if e.dxftype() in dxf_dict[e.dxf.layer]:
                dxf_dict[e.dxf.layer][e.dxftype()].append(points)
            else:
                dxf_dict[e.dxf.layer][e.dxftype()] = [points]
        else:
            dxf_dict[e.dxf.layer] = {e.dxftype(): [points]}
    return dxf_dict

In [4]:
layers_data = {}
for path in glob.iglob(thesis_path + r"\Data\\maps\\dxf\\" + "*.dxf"): # alt: with open("data/layers.json") as f: l = json.load(f)
    layers_data.update(dxf_to_dict(path))

In [5]:
polygons = {}
for i, polygon in enumerate(layers_data['Twrsysrules Rode lijnen']['LWPOLYLINE']):
    polygons['red_zone_{:0>2d}'.format(i)] = np.array(polygon).round(3).tolist()

In [5]:
# Alternative Polygons:
polygons = []
for i, polygon in enumerate(layers_data['Twrsysrules Rode lijnen']['LWPOLYLINE']):
    polygons.append(np.array(polygon).round(3).tolist())

In [7]:
with open(thesis_path + r"\Results\\" + "Red_Zone_Polygons\\" + "red_zone_polygons.json", 'w') as f:
    json.dump(polygons, f, indent=4)

In [8]:
# Alternative Polygons:
with open(thesis_path + "\\Results\\" + "Red_Zone_Polygons\\" + "red_zone_polygons_alter.json", 'w') as f:
    json.dump({"red_zone": polygons}, f, indent=4)